In [38]:
from model import SentimentBERT
import pandas as pd
import zipfile
import os
import torch

In [2]:
zip_file_path = 'model.zip'  
output_directory = './model'   



def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        zipf.extractall(extract_to)
        print(f'Файлы из "{zip_path}" были успешно извлечены в "{extract_to}".')

os.makedirs(output_directory, exist_ok=True)
unzip_file(zip_file_path, output_directory)

model = SentimentBERT(output_directory)

Файлы из "model.zip" были успешно извлечены в "./model".


In [3]:
text = '''я тебя ненавижу'''
model(text)

{'class': 'negative', 'probabilities': tensor([[0.0707, 0.0090, 0.9203]])}

In [16]:
text = '''я тебя ненавижу (почти)'''
model(text)

{'class': 'negative', 'probabilities': tensor([[0.2157, 0.0133, 0.7709]])}

In [17]:
text = '''я возможно тебя ненавижу (почти)'''
model(text)

{'class': 'negative', 'probabilities': tensor([[0.3646, 0.0142, 0.6212]])}

In [19]:
text = '''я возможно тебя ненавижу (почти), но все равно ты хороший'''
model(text)

{'class': 'neutral', 'probabilities': tensor([[0.5768, 0.1619, 0.2612]])}

In [20]:
text = '''Путин победил на выборах президента'''
model(text)

{'class': 'neutral', 'probabilities': tensor([[0.8931, 0.0916, 0.0153]])}

In [30]:
text = '''Ура! Путин победил на выборах президента'''
model(text)

{'class': 'positive', 'probabilities': tensor([[0.0467, 0.9508, 0.0025]])}

In [5]:
news = pd.read_csv("./data/outp.csv")
news.head()

,Unnamed: 0,title,description,link,published
0,0,Австралийский теннисист резко высказался о мат...,NaN,https://lenta.ru/news/2024/12/15/avstraliyskiy...,"Sun, 15 Dec 2024 23:55:28 +0300"
1,1,В Турции заявили о нежелании тратить десятки л...,NaN,https://lenta.ru/news/2024/12/15/v-turtsii-zay...,"Sun, 15 Dec 2024 23:41:00 +0300"
2,2,В России выбрали культурную столицу 2026 года,NaN,https://lenta.ru/news/2024/12/15/v-rossii-vybr...,"Sun, 15 Dec 2024 23:36:00 +0300"
3,3,Советник рассказал о контактах Трампа с против...,NaN,https://lenta.ru/news/2024/12/15/sovetnik-rass...,"Sun, 15 Dec 2024 23:34:00 +0300"
4,4,В Польше раскрыли подробности санкций ЕС за «г...,NaN,https://lenta.ru/news/2024/12/15/v-polshe-soob...,"Sun, 15 Dec 2024 23:24:00 +0300"


In [52]:
processed_news = []
for _, row in news.iterrows():
    lb = model(row["title"])
    processed_news.append({"title": row["title"], 
                           "link": row["link"], 
                           "class": lb["class"],
                           "logit": [round(float(i), 2) for i in lb["probabilities"][0]]})

In [56]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('news_template.html')

# Генерация HTML
html_output = template.render(news=processed_news)

# Сохранение HTML в файл
with open('news.html', 'w', encoding='utf-8') as f:
    f.write(html_output)

print("HTML файл успешно сгенерирован!")

HTML файл успешно сгенерирован!
